### This code uploads all contents of a folder to a repository in hugging face. If your folder has hundreds of files, expect upload failures (and many of them). In case of failures, just wait a bit and run this code again. Repeat until the code executes without failing. We upload by parts (n files per commit), because uploading everything at once can be difficult.

In [1]:
#Change below:
folder="AbdomenAtlas/AbdomenAtlasMini1.0"#path on your computer
repository="AbdomenAtlas/AbdomenAtlas_1.0_Mini"#Hugging face repository
repository_type="dataset" #dataset, model or space
n=100#number of files per commit

In [1]:
#login if needed
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
import huggingface_hub
from pathlib import Path
from huggingface_hub import HfApi, CommitOperationAdd

In [3]:
def get_all_files(root: Path):
    dirs = [root]
    while len(dirs) > 0:
        dir = dirs.pop()
        for candidate in dir.iterdir():
            if candidate.is_file():
                yield candidate
            if candidate.is_dir():
                dirs.append(candidate)

def get_groups_of_n(n: int, iterator):
    assert n > 1
    buffer = []
    for elt in iterator:
        if len(buffer) == n:
            yield buffer
            buffer = []
        buffer.append(elt)
    if len(buffer) != 0:
        yield buffer


In [5]:
api = HfApi()
root = Path(folder)

try:
    counter=torch.load('counter.pt')
except:
    counter=0
#counter=0

for i, file_paths in enumerate(get_groups_of_n(n, get_all_files(root))):
    if i<counter:
        continue
    print(f"Committing {i}")
    operations = [
        CommitOperationAdd(path_in_repo='/'.join(str(file_path).split('/')[2:]), 
                           path_or_fileobj=str(file_path))
        for file_path in file_paths
    ]
    api.create_commit(
        repo_id=repository,
        operations=operations,
        commit_message=f"Upload part {i}",
        repo_type=repository_type
    )
    torch.save(i,'counter.pt')


Committing 519
